# Pruebas con barrido de frecuencia

In [1]:
import threading
import time
import numpy as np
import sounddevice as sd
import serial
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import matplotlib.pyplot as plt
from ordered_set import OrderedSet

In [ ]:
import sys
sys.path.append('..')
# Custom imports
from analysis_utils import DataContainer
from audio_constants import ref_volume, freq_response_waves, freq_sweep_waves, freq_weighting_waves

In [2]:
ESP_COM_PORT = "COM23"
ESP_BAUDRATE = 115200

In [4]:
# Get the default audio playback device
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)

# Create a volume control interface
volume_controller = cast(interface, POINTER(IAudioEndpointVolume))

In [5]:
# Variables for status syncronization between threads
current_freq = 0
current_volume = 0.0

In [15]:
# Function to receive serial data and store it
def receive_serial_data(data):
    ser = serial.Serial(ESP_COM_PORT, ESP_BAUDRATE)  # Replace 'COM1' with your serial port
    while not terminate_event.is_set():
        received_data = ser.readline().decode().strip()
        received_data = received_data.split(" ")[0]
        try:
            received_data = float(received_data)
        except ValueError:
            received_data = float(received_data.split(".")[0])
        try:
            if log_flag:
                data[current_freq][current_volume] = np.append(data[current_freq][current_volume], received_data)
            if freq_flag:
                data[current_freq]["freq_all"] = np.append(data[current_freq]["freq_all"], received_data)
            data["all"] = np.append(data["all"], received_data)
        except KeyError:
            pass
    ser.close()

# Function to play audio sine waves
def play_test_waves(test_waves):
    global current_freq, current_volume, log_flag, freq_flag

    duration = 2.5 if TIME_PERIOD == 0.125 else 20
    sample_rate = 88200
    t = np.arange(0, duration, 1 / sample_rate)

    for freq in test_waves["frequencies"]:
        current_freq = freq
        sine_wave = np.sin(2 * np.pi * freq * t)

        for n, volume in enumerate(test_waves["volumes"]):
            current_volume = volume
            print(f"now playing: {volume} volume @ {freq} Hz")

            log_flag = True
            if not freq_flag:
                freq_flag = True

            volume_controller.SetMasterVolumeLevelScalar(volume/10, None)

            # Wait before playing the sine wave
            time.sleep(0.375 if TIME_PERIOD == 0.125 else 3) #s

            sd.play(sine_wave, sample_rate, blocking=True)

            # Wait after playback is done
            time.sleep(0.375 if TIME_PERIOD == 0.125 else 3) #s

            log_flag = False
            if n == (len(test_waves["volumes"]) - 1):
                freq_flag = False

    # Send a signal to the other thread to terminate
    terminate_event.set()

In [8]:
# Doesn't sync the update times of the SLM and ESP, so it's not necessary
def play_sync_impulses():
    # Duration of the impulse in seconds
    duration = 0.375 if TIME_PERIOD == 0.125 else 3 # s
    sample_rate = 88200
    t = np.arange(0, duration, 1 / sample_rate)

    # Create an impulse signal
    impulse = np.sin(2 * np.pi * 1000 * t)

    # Wait before playing the impulse
    time.sleep(0.375 if TIME_PERIOD == 0.125 else 3) #s

    # Play the impulse
    sd.play(impulse, sample_rate, blocking=True)

    time.sleep(0.3 if TIME_PERIOD == 0.125 else 2)

    sd.play(impulse, sample_rate, blocking=True)

    time.sleep(0.6 if TIME_PERIOD == 0.125 else 4)

    sd.play(impulse, sample_rate, blocking=True)

    time.sleep(0.375 if TIME_PERIOD == 0.125 else 3)

In [16]:
# Create formatted containers to store received serial data
freq_response_data = DataContainer(freq_response_waves)
freq_sweep_data_F = DataContainer(freq_sweep_waves)
freq_sweep_data_S = DataContainer(freq_sweep_waves)
freq_weighting_data = DataContainer(freq_weighting_waves)

# Event to signal termination
terminate_event = threading.Event()
# Flag to signal when to log measurements
log_flag = False
# Flag to signal the duration of a level progression for a certain frequency
freq_flag = False

## Ejecutar pruebas

### Respuesta de frecuencia y nivel

In [ ]:
TIME_PERIOD = 0.125

# *** Frequency response ***

# Create the serial data receiving thread
serial_thread = threading.Thread(target=receive_serial_data, args=(freq_response_data,), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_test_waves, args=(freq_response_waves,), daemon=True)

# Start serial thread
serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
serial_thread.join()
audio_thread.join()

# Reset the event for next test
terminate_event.clear()

### Barrido de frecuencia y nivel con periodo F

In [17]:
TIME_PERIOD = 0.125

# Create the serial data receiving thread
serial_thread = threading.Thread(target=receive_serial_data, args=(freq_sweep_data_F,), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_test_waves, args=(freq_sweep_waves,), daemon=True)

# Start serial thread
serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
serial_thread.join()
audio_thread.join()

# Reset the event for next test
terminate_event.clear()

now playing: 0.03 volume @ 1000 Hz
now playing: 0.04 volume @ 1000 Hz
now playing: 0.05 volume @ 1000 Hz
now playing: 0.03 volume @ 2000 Hz
now playing: 0.04 volume @ 2000 Hz
now playing: 0.05 volume @ 2000 Hz
now playing: 0.03 volume @ 3000 Hz
now playing: 0.04 volume @ 3000 Hz
now playing: 0.05 volume @ 3000 Hz


### Barrido de frecuencia y nivel con periodo S

In [ ]:
TIME_PERIOD = 1.0

# Create the serial data receiving thread
serial_thread = threading.Thread(target=receive_serial_data, args=(freq_sweep_data_S,), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_test_waves, args=(freq_sweep_waves,), daemon=True)

# Start serial thread
serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
serial_thread.join()
audio_thread.join()

# Reset the event for next test
terminate_event.clear()

### Ponderación de frecuencia

In [ ]:
TIME_PERIOD = 0.125

# Create the serial data receiving thread
serial_thread = threading.Thread(target=receive_serial_data, args=(freq_weighting_data,), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_test_waves, args=(freq_weighting_waves,), daemon=True)

# Start serial thread
serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
serial_thread.join()
audio_thread.join()

# Reset the event for next test
terminate_event.clear()

## Guardar datos

In [ ]:
with open("../results/sweep_tests/freq_response.pkl", 'wb') as file:
    pickle.dump(freq_response_data, file)

with open("../results/sweep_tests/freq_sweep_f.pkl", 'wb') as file:
    pickle.dump(freq_sweep_data_F, file)

with open("../results/sweep_tests/freq_sweep_s.pkl", 'wb') as file:
    pickle.dump(freq_sweep_data_S, file)

with open("../results/sweep_tests/freq_wting.pkl", 'wb') as file:
    pickle.dump(freq_weighting_data, file)